# 100_fig10_sensitivity_p21

Figure 10: sensitivity curves by varying p21 and recomputing Taylor SSS inflation and real rate.

In [ ]:
import os, sys, json, numpy as np, torch, matplotlib.pyplot as plt
sys.path.append("..")

from src.config import ModelParams
from src.io_utils import load_json, load_npz, load_torch, load_selected_run, find_latest_run_dir
from src.deqn import PolicyNetwork

ART = os.path.join("..","artifacts","runs")

def get_run(policy: str) -> str:
    rd = load_selected_run(ART, policy)
    if rd is None:
        rd = find_latest_run_dir(ART, policy)
    if rd is None:
        raise RuntimeError(f"No runs found for policy={policy} under {ART}")
    return rd

def _parse_dtype(s: str):
    if s is None:
        return torch.float32
    if isinstance(s, torch.dtype):
        return s
    s = str(s)
    if "float64" in s:
        return torch.float64
    if "float32" in s:
        return torch.float32
    if "bfloat16" in s:
        return torch.bfloat16
    return torch.float32

def load_params_from_run(run_dir: str, *, device="cpu"):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    p = cfg.get("params", {})
    dtype = _parse_dtype(p.get("dtype"))
    dev = device if device is not None else p.get("device","cpu")
    keep = {k:v for k,v in p.items() if k in ModelParams.__dataclass_fields__}
    keep["device"] = dev
    keep["dtype"] = dtype
    return ModelParams(**keep).to_torch()

def load_net_from_run(run_dir: str, d_in: int, d_out: int):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    tc = cfg.get("train_cfg", {})
    hidden = tuple(tc.get("hidden_layers", (512,512)))
    activation = tc.get("activation", "selu")
    net = PolicyNetwork(d_in, d_out, hidden=hidden, activation=activation)
    state = load_torch(os.path.join(run_dir, "weights.pt"), map_location="cpu")
    # state is usually a plain state_dict
    if isinstance(state, dict) and "state_dict" in state:
        state = state["state_dict"]
    net.load_state_dict(state)
    net.eval()
    return net

In [ ]:
from src.steady_states import solve_flexprice_sss, solve_taylor_sss

ann = lambda x: 400.0 * x
params0 = ModelParams(device="cpu", dtype=torch.float32)

grid = np.linspace(0.02, 0.5, 25)
pi_bad = []
pi_norm = []
r_bad = []
r_norm = []

for p21 in grid:
    params = ModelParams(
        beta=params0.beta, gamma=params0.gamma, omega=params0.omega, theta=params0.theta, eps=params0.eps, tau_bar=params0.tau_bar,
        rho_A=params0.rho_A, rho_tau=params0.rho_tau, rho_g=params0.rho_g,
        sigma_A=params0.sigma_A, sigma_tau=params0.sigma_tau, sigma_g=params0.sigma_g,
        g_bar=params0.g_bar, eta_bar=params0.eta_bar, bad_state=params0.bad_state,
        p12=params0.p12, p21=float(p21),
        pi_bar=params0.pi_bar, psi=params0.psi,
        device="cpu", dtype=torch.float32
    ).to_torch()
    flex = solve_flexprice_sss(params)
    tay = solve_taylor_sss(params, flex)
    pi_norm.append(float(tay.by_regime[0]["pi"]))
    pi_bad.append(float(tay.by_regime[1]["pi"]))
    r_norm.append(float(tay.by_regime[0]["r"]))
    r_bad.append(float(tay.by_regime[1]["r"]))

fig, ax = plt.subplots(1, 2, figsize=(11, 4))

ax[0].plot(grid, ann(np.asarray(pi_norm)), label="pi_sss normal")
ax[0].plot(grid, ann(np.asarray(pi_bad)), label="pi_sss bad")
ax[0].set_title("Figure 10a: Taylor SSS inflation vs p21")
ax[0].set_xlabel("p21 (bad->normal probability)")
ax[0].set_ylabel("Annualized percent")
ax[0].legend()

ax[1].plot(grid, ann(np.asarray(r_norm)), label="r_sss normal")
ax[1].plot(grid, ann(np.asarray(r_bad)), label="r_sss bad")
ax[1].set_title("Figure 10b: Taylor SSS real rate vs p21")
ax[1].set_xlabel("p21 (bad->normal probability)")
ax[1].set_ylabel("Annualized percent")
ax[1].legend()

plt.tight_layout()
plt.show()
